# Blue Carbon Cost Tool Coun

In this notebook, we will add GDAM geometries to the Blue Carbon Cost Tool countries.

## Setup

### Library import

In [1]:
import geopandas as gpd
import pandas as pd

## Read data

In [2]:
data_path = "../excel/Carbon-Cost Data Upload v02.xlsm"
df = pd.read_excel(data_path, sheet_name="Countries")
df = df[["country", "country_code"]]
df.head()

,country,country_code
0,Afghanistan,AFG
1,Åland Islands,ALA
2,Albania,ALB
3,Algeria,DZA
4,American Samoa,ASM


In [3]:
gdf_adm0 = gpd.read_file("../raw_data/gadm_410-adm0.json")
gdf_adm0.rename(columns={"COUNTRY": "country", "GID_0": "country_code"}, inplace=True)
gdf_adm0 = gdf_adm0[["country","country_code", "geometry"]]
gdf_adm0.head()

,country,country_code,geometry
0,Aruba,ABW,"POLYGON ((-69.9782 12.46986, -69.97014 12.4659..."
1,Afghanistan,AFG,"POLYGON ((60.89944 29.8375, 60.90917 29.83284,..."
2,Angola,AGO,"MULTIPOLYGON (((11.75755 -17.25208, 11.77201 -..."
3,Anguilla,AIA,"MULTIPOLYGON (((-63.02064 18.2075, -63.01217 1..."
4,Åland,ALA,"MULTIPOLYGON (((21.16194 59.7793, 21.14583 59...."


In [4]:
gdf_adm1 = gpd.read_file("../raw_data/gadm_410-adm1.json")
gdf_adm1.rename(columns={"GID_0": "country_code", "NAME_1": "country"}, inplace=True)
gdf_adm1 = gdf_adm1[["country", "geometry"]]
gdf_adm1 = gdf_adm1[gdf_adm1["country"].isin(["Hong Kong", "Macau"])]
gdf_adm1["country_code"] = ["HKG", "MAC"]
gdf_adm1 = gdf_adm1[["country", "country_code", "geometry"]]
gdf_adm1

,country,country_code,geometry
498,Hong Kong,HKG,"MULTIPOLYGON (((114.25181 22.16125, 114.25792 ..."
505,Macau,MAC,"MULTIPOLYGON (((113.54885 22.13993, 113.55153 ..."


In [5]:
gdf = pd.concat([gdf_adm0, gdf_adm1], ignore_index=True)
df_new = df.merge(gdf[["country_code", "geometry"]], on="country_code", how="left")
df_new.head()

,country,country_code,geometry
0,Afghanistan,AFG,"POLYGON ((60.89944 29.8375, 60.90917 29.83284,..."
1,Åland Islands,ALA,"MULTIPOLYGON (((21.16194 59.7793, 21.14583 59...."
2,Albania,ALB,"MULTIPOLYGON (((20.00848 39.694, 20.01152 39.6..."
3,Algeria,DZA,"MULTIPOLYGON (((8.64567 36.93663, 8.64142 36.9..."
4,American Samoa,ASM,"MULTIPOLYGON (((-170.68064 -14.28248, -170.683..."


## Save data

In [6]:
gpd.GeoDataFrame(df_new).to_file("../raw_data/countries.geojson", driver="GeoJSON")